In [1]:
!pip install -q gradio
!pip install python-multipart
!pip install transformers
!pip install torch


In [2]:

import gradio as gr
import random
import time
from gradio import Image
import torch
from transformers import BertForQuestionAnswering, BertTokenizerFast
from google.colab import drive
drive.mount('/content/drive')

# Load the pre-trained model and tokenizer
model_path = '/content/drive/MyDrive/saved_model/'
model = BertForQuestionAnswering.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)
image = Image(value="/content/drive/MyDrive/logo.jpeg", width=150, height=150, show_label=False)
def answer_question(question, context):
    # Tokenize input
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True, max_length=512)

    # Generate output
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the most likely beginning and end of the answer span
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Find the tokens with the highest `start` and `end` scores
    answer_start = torch.argmax(answer_start_scores)  # Get the most likely start of the answer
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of the answer

    # Convert the tokens to text
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    answer=answer[0].upper()+ answer[1:len(answer)]

    return answer



with gr.Blocks(title="Our Chatbot", css=".dialog-box {border: 1px solid #ccc; border-radius: 8px; padding: 20px; background-color:black;} .header-box { border: 1px solid #ccc; background-color: white; } " ) as demo:
    with gr.Group(elem_classes="dialog-box"):
        with gr.Row(elem_classes="header-box" ):
            gr.Markdown("""  #       <h1 style="color:red;"> &ensp; Info Geine</h1>
            <p> &ensp; AAI-520 NLP Final Project Chatbot</p>
            <p> &ensp; Group Member: <b>Aditya Ranjeet and Rajesh</b></p>""")
            image.render()

        with gr.Row(elem_classes="header-box"):
            context = gr.Textbox(label="Context", elem_classes="1header-box")
            chatbot = gr.Chatbot(type="messages", layout = 'bubble',min_width= 640 )
        msg = gr.Textbox(label="Ask your Question here")
        clear = gr.ClearButton([msg, chatbot,context])

    def respond(message, chat_history,context):

        bot_message = answer_question(message, context)
        chat_history.append({"role": "user", "content": message})
        chat_history.append({"role": "assistant", "content": bot_message})
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot,context], [msg, chatbot])

demo.launch(share=True)

/usr/local/lib/python3.10/dist-packages/starlette/formparsers.py:12: FutureWarning: Please use `import python_multipart` instead.
  import multipart


Mounted at /content/drive
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2eb9a5f010f68f9451.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
